In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('n50.csv')

In [3]:
def number_of_years(y):  # calculates the number of years of the dataset
    p = y.index[0]  # date of first row in the dataset (datetime format)
    q = y.index[len(y) - 1]  # date of last row in the dataset  (datetime format)
    return ((q - p).days + 1) / 365  # the difference give the number of total days (not trading days) over the total number of years in the dataset

In [4]:
def ratio(a, b, c):  # function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    return (a - c) / b  # a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)


In [5]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights


In [6]:
mvo_sortino=pd.DataFrame(columns=['Date', 'Returns', 'Risk','Ratio'])
inputweights=[]

In [7]:
def MVO_Sortino(start_date,end_date,mvo_sortino):
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=60
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0  
    covmatso = returnso.cov() * trading_days
    num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
    num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358/364
    portfolio_returns = []                    #initializing an empty list for portfolio returns
    portfolio_volatility =[]                  #initializing an empty list for portfolio risk
    stock_weights =[]                         #initializing an empty list for portfolio weights
    sortino =[]          
    for single_portfolio in range(5):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
        varso=np.dot(weights.T,np.dot(covmatso,weights))
        semi_temp = np.sqrt(varso)
        sotemp = ratio(returns_temp,semi_temp,risk_free_rate)   #calculating sharpe ratio
        portfolio_returns.append(returns_temp)                       
        portfolio_volatility.append(semi_temp)
        stock_weights.append(weights)
        sortino.append(sotemp)
    portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Sortino Ratio' : sortino}
    for counter,symbol in enumerate(df.columns):
        portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights]
    pc = pd.DataFrame(portfolio)
    pc.loc[:, :] *= 100
    pc['Sortino Ratio']=pc['Sortino Ratio']/100 
    max_sortino=pc['Sortino Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
    max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
    max_weights=max_sortino_portfolio.values
    max_weights=max_weights[0][3:]
    return max_weights

In [8]:
def MVO_Sortino_lookback(start_date,end_date,deciding_date,mvo_sortino,inputweights):
    mvo_sortino_list=[]
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=60
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0  
    covmatsh = returnsh.cov() * trading_days  # Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
    covmatso = returnso.cov() * trading_days  # Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio
    num_assets = len(df.columns)  # initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358  # initializing risk free rate that will be used in calculating both the ratios (absolute value)
# referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
# In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

# 2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
# Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

    global_war_sortino=[]
    sortino_portfolio_return=[]
    sortino_portfolio_risk=[]
    sortino_portfolio_soratio=[]
    sortino_portfolio_stockWeights=[]
    fitness=0
    weights=np.array(BALANCE(np.array(inputweights)))
    returns_temp = np.sum(returnso.mean()*weights)*trading_days 
    varso=np.dot(weights.T,np.dot(covmatso,weights))   
    semi_temp = np.sqrt(varso)      
    fitness = ratio(returns_temp,semi_temp,risk_free_rate)
    sortino_portfolio_return.append(returns_temp)
    sortino_portfolio_risk.append(semi_temp)
    sortino_portfolio_soratio.append(fitness)
    sortino_portfolio_stockWeights.append(weights)
    sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  
    for counter,symbol in enumerate(df.columns):
      sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
    sortino_pc = pd.DataFrame(sortino_portfolio)
    sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
    sortino_optimal=sortino_optimal.to_frame()
    sortino_optimal=sortino_optimal.transpose()
    sortino_optimal.loc[:, :] *= 100
    sortino_optimal.loc[:, 'Sortino Ratio'] /= 100
    to_append=[deciding_date,sortino_optimal['Returns'].values[0],sortino_optimal['Semi Deviation'].values[0],sortino_optimal['Sortino Ratio'].values[0]]
    mvo_sortino_series = pd.Series(to_append, index = mvo_sortino.columns)
    mvo_sortino = mvo_sortino.append(mvo_sortino_series, ignore_index=True)
    sortino_optimal.to_csv(str(deciding_date)+"sortino_optimal.csv")
    print(mvo_sortino)
    return mvo_sortino


In [9]:
start_date="2016-01-01"

In [10]:
start_date_index=df.index[df['Date'] == start_date]

In [11]:
end_date="2020-09-30"

In [12]:
end_date_index=df.index[df['Date']== end_date]

In [13]:
start_date_lookback="2020-10-01"

In [14]:
start_date_lookback_index=df.index[df['Date'] == start_date_lookback]

In [15]:
end_date_lookback="2020-12-31"

In [16]:
end_date_lookback_index=df.index[df['Date']== end_date_lookback]


In [17]:
while df.iloc[end_date_index].Date.values[0]:
    inputweights=MVO_Sortino(df.iloc[start_date_index].Date.values[0],df.iloc[end_date_index].Date.values[0],mvo_sortino)
    mvo_sortino=MVO_Sortino_lookback(df.iloc[start_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index+1].Date.values[0],mvo_sortino,inputweights)
    start_date_lookback_index+=1
    end_date_lookback_index+=1
    start_date_index+=1
    end_date_index+=1

         Date    Returns     Risk     Ratio
0  2021-01-01 -29.511061  3.79331 -8.723533
         Date    Returns      Risk     Ratio
0  2021-01-01 -29.511061  3.793310 -8.723533
1  2021-01-04 -31.577649  4.462152 -7.879080
         Date    Returns      Risk     Ratio
0  2021-01-01 -29.511061  3.793310 -8.723533
1  2021-01-04 -31.577649  4.462152 -7.879080
2  2021-01-05 -30.248611  4.065760 -8.320366
         Date    Returns      Risk     Ratio
0  2021-01-01 -29.511061  3.793310 -8.723533
1  2021-01-04 -31.577649  4.462152 -7.879080
2  2021-01-05 -30.248611  4.065760 -8.320366
3  2021-01-06 -31.636337  4.365513 -8.066942
         Date    Returns      Risk     Ratio
0  2021-01-01 -29.511061  3.793310 -8.723533
1  2021-01-04 -31.577649  4.462152 -7.879080
2  2021-01-05 -30.248611  4.065760 -8.320366
3  2021-01-06 -31.636337  4.365513 -8.066942
4  2021-01-07 -29.512807  4.203623 -7.872450
         Date    Returns      Risk     Ratio
0  2021-01-01 -29.511061  3.793310 -8.723533
1  2021-01-0

          Date    Returns      Risk     Ratio
0   2021-01-01 -29.511061  3.793310 -8.723533
1   2021-01-04 -31.577649  4.462152 -7.879080
2   2021-01-05 -30.248611  4.065760 -8.320366
3   2021-01-06 -31.636337  4.365513 -8.066942
4   2021-01-07 -29.512807  4.203623 -7.872450
5   2021-01-08 -30.652697  3.903527 -8.769683
6   2021-01-11 -30.999945  4.232104 -8.170864
7   2021-01-12 -32.474898  4.401142 -8.192169
8   2021-01-13 -31.059831  4.197178 -8.253125
9   2021-01-14 -29.156162  3.890656 -8.414047
10  2021-01-15 -28.362940  3.882231 -8.227985
11  2021-01-18 -31.013731  4.174599 -8.286720
12  2021-01-19 -31.495349  4.289110 -8.177769
13  2021-01-20 -30.278572  4.186009 -8.088510
14  2021-01-21 -29.613206  3.909703 -8.489957
15  2021-01-22 -30.945861  4.422659 -7.806584
16  2021-01-25 -30.591389  4.062821 -8.410754
17  2021-01-27 -31.035800  4.245834 -8.152887
18  2021-01-28 -32.547932  4.519423 -7.993925
          Date    Returns      Risk     Ratio
0   2021-01-01 -29.511061  3.79331

          Date    Returns      Risk     Ratio
0   2021-01-01 -29.511061  3.793310 -8.723533
1   2021-01-04 -31.577649  4.462152 -7.879080
2   2021-01-05 -30.248611  4.065760 -8.320366
3   2021-01-06 -31.636337  4.365513 -8.066942
4   2021-01-07 -29.512807  4.203623 -7.872450
5   2021-01-08 -30.652697  3.903527 -8.769683
6   2021-01-11 -30.999945  4.232104 -8.170864
7   2021-01-12 -32.474898  4.401142 -8.192169
8   2021-01-13 -31.059831  4.197178 -8.253125
9   2021-01-14 -29.156162  3.890656 -8.414047
10  2021-01-15 -28.362940  3.882231 -8.227985
11  2021-01-18 -31.013731  4.174599 -8.286720
12  2021-01-19 -31.495349  4.289110 -8.177769
13  2021-01-20 -30.278572  4.186009 -8.088510
14  2021-01-21 -29.613206  3.909703 -8.489957
15  2021-01-22 -30.945861  4.422659 -7.806584
16  2021-01-25 -30.591389  4.062821 -8.410754
17  2021-01-27 -31.035800  4.245834 -8.152887
18  2021-01-28 -32.547932  4.519423 -7.993925
19  2021-01-29 -32.303468  4.455873 -8.053072
20  2021-02-01 -32.621354  4.63476

          Date    Returns      Risk     Ratio
0   2021-01-01 -29.511061  3.793310 -8.723533
1   2021-01-04 -31.577649  4.462152 -7.879080
2   2021-01-05 -30.248611  4.065760 -8.320366
3   2021-01-06 -31.636337  4.365513 -8.066942
4   2021-01-07 -29.512807  4.203623 -7.872450
5   2021-01-08 -30.652697  3.903527 -8.769683
6   2021-01-11 -30.999945  4.232104 -8.170864
7   2021-01-12 -32.474898  4.401142 -8.192169
8   2021-01-13 -31.059831  4.197178 -8.253125
9   2021-01-14 -29.156162  3.890656 -8.414047
10  2021-01-15 -28.362940  3.882231 -8.227985
11  2021-01-18 -31.013731  4.174599 -8.286720
12  2021-01-19 -31.495349  4.289110 -8.177769
13  2021-01-20 -30.278572  4.186009 -8.088510
14  2021-01-21 -29.613206  3.909703 -8.489957
15  2021-01-22 -30.945861  4.422659 -7.806584
16  2021-01-25 -30.591389  4.062821 -8.410754
17  2021-01-27 -31.035800  4.245834 -8.152887
18  2021-01-28 -32.547932  4.519423 -7.993925
19  2021-01-29 -32.303468  4.455873 -8.053072
20  2021-02-01 -32.621354  4.63476

          Date    Returns      Risk     Ratio
0   2021-01-01 -29.511061  3.793310 -8.723533
1   2021-01-04 -31.577649  4.462152 -7.879080
2   2021-01-05 -30.248611  4.065760 -8.320366
3   2021-01-06 -31.636337  4.365513 -8.066942
4   2021-01-07 -29.512807  4.203623 -7.872450
5   2021-01-08 -30.652697  3.903527 -8.769683
6   2021-01-11 -30.999945  4.232104 -8.170864
7   2021-01-12 -32.474898  4.401142 -8.192169
8   2021-01-13 -31.059831  4.197178 -8.253125
9   2021-01-14 -29.156162  3.890656 -8.414047
10  2021-01-15 -28.362940  3.882231 -8.227985
11  2021-01-18 -31.013731  4.174599 -8.286720
12  2021-01-19 -31.495349  4.289110 -8.177769
13  2021-01-20 -30.278572  4.186009 -8.088510
14  2021-01-21 -29.613206  3.909703 -8.489957
15  2021-01-22 -30.945861  4.422659 -7.806584
16  2021-01-25 -30.591389  4.062821 -8.410754
17  2021-01-27 -31.035800  4.245834 -8.152887
18  2021-01-28 -32.547932  4.519423 -7.993925
19  2021-01-29 -32.303468  4.455873 -8.053072
20  2021-02-01 -32.621354  4.63476

...

           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
260  2022-01-19 -37.768259  4.667959 -8.857888
261  2022-01-20 -36.128243  4.421227 -8.981272
262  2022-01-21 -37.499399  4.393123 -9.350842
263  2022-01-24 -37.085503  4.589200 -8.861130
264  2022-01-25 -40.072259  5.427828 -8.042306

[265 rows x 4 columns]
           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
261  2022-01-20 -36.128243  4.421227 -8.981272
262  2022-01-21 -37.499399  4.393123

           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
274  2022-02-09 -40.232399  5.620549 -7.795038
275  2022-02-10 -38.278884  5.142493 -8.139803
276  2022-02-11 -39.301461  5.456774 -7.858390
277  2022-02-14 -40.587259  5.655193 -7.810035
278  2022-02-15 -43.241228  6.068436 -7.715534

[279 rows x 4 columns]
           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
275  2022-02-10 -38.278884  5.142493 -8.139803
276  2022-02-11 -39.301461  5.456774

           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
288  2022-03-02 -40.771457  6.714556 -6.605270
289  2022-03-03 -42.469293  6.723125 -6.849388
290  2022-03-04 -43.803741  7.238990 -6.545629
291  2022-03-07 -42.674775  6.901956 -6.701691
292  2022-03-08 -44.263892  7.159352 -6.682712

[293 rows x 4 columns]
           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
289  2022-03-03 -42.469293  6.723125 -6.849388
290  2022-03-04 -43.803741  7.238990

           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
302  2022-03-23 -42.588335  7.076967 -6.523746
303  2022-03-24 -42.876690  6.936104 -6.697808
304  2022-03-25 -43.101929  7.025883 -6.644279
305  2022-03-28 -43.229814  7.403569 -6.322601
306  2022-03-29 -43.218915  6.793455 -6.888824

[307 rows x 4 columns]
           Date    Returns      Risk     Ratio
0    2021-01-01 -29.511061  3.793310 -8.723533
1    2021-01-04 -31.577649  4.462152 -7.879080
2    2021-01-05 -30.248611  4.065760 -8.320366
3    2021-01-06 -31.636337  4.365513 -8.066942
4    2021-01-07 -29.512807  4.203623 -7.872450
..          ...        ...       ...       ...
303  2022-03-24 -42.876690  6.936104 -6.697808
304  2022-03-25 -43.101929  7.025883

IndexError: positional indexers are out-of-bounds

In [ ]:
mvo_sortino.to_csv('mvo_sortino.csv')